# Linear Regression on YearPredictionMSD

In this notebook we are going to train a linear regression model on the YearPredictionMSD, a data set that contains features extracted from songs. The model that we will train is going to predict which year a song was from.

## 1. Instantiate ZipML_SGD

First, we instantiate a ZipML_SGD object, that we will use for training and inference.

In [1]:
import zipml
import numpy as np
import time

Z = zipml.ZipML_SGD(on_pynq=1, bitstreams_path=zipml.BITSTREAMS, ctrl_base=0x41200000, dma_base=0x40400000, dma_buffer_size=32*1024*1024)

floatFSGD.bit is loaded
Got sgd_ctrl!
Got sgd_dma!
Allocated buffer of size: 33554432 bytes


## 2. Load and prepare the data

We load the data that is formatted in libsvm format (label feature_index1:feature1 feature_index2:feature2 ...). Then, we perform a normalization on the features of the data set, where we can specify: (1) The normalization range (-1 to 1 or 0 to 1), (2) if the normalization should happen row-wise or column-wise.

In [3]:
!./get_yearpredictionMSD.sh

--2017-10-30 09:21:17--  https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/regression/YearPredictionMSD.bz2
Resolving www.csie.ntu.edu.tw (www.csie.ntu.edu.tw)... 140.112.30.26
Connecting to www.csie.ntu.edu.tw (www.csie.ntu.edu.tw)|140.112.30.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 212177525 (202M) [application/x-bzip2]
Saving to: ‘YearPredictionMSD.bz2’

YearPredictionMSD.b 100%[=====================>] 202.35M  3.19MB/s   in 40s    

2017-10-30 09:21:58 (5.11 MB/s) - ‘YearPredictionMSD.bz2’ saved [212177525/212177525]

rm: cannot remove ‘YearPredictionMSD.bz2’: No such file or directory
--2017-10-30 09:26:33--  https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/regression/YearPredictionMSD.t.bz2
Resolving www.csie.ntu.edu.tw (www.csie.ntu.edu.tw)... 140.112.30.26
Connecting to www.csie.ntu.edu.tw (www.csie.ntu.edu.tw)|140.112.30.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23639653 (23M) [application/x-bz

In [4]:
start = time.time()
Z.load_libsvm_data('./YearPredictionMSD/YearPredictionMSD', 50000, 90)
print('a loaded, time: ' + str(time.time()-start) )
Z.a_normalize(to_min1_1=1, row_or_column='c');
Z.b_normalize(to_min1_1=0)

# Set training related parameters
num_epochs = 10
step_size = 1.0/(1 << 12)

num_values_one_row: 94
a loaded, time: 1150.6679298877716


## 3. Full precision training using the CPU

Let's perform training on full-precision data using the CPU. During the training, we calculate the loss after each epoch (a complete scan over the data set). The convergence of the model is observed by the decreasing loss. After having performed the given number of epochs, we save the trained model in an array.

In [5]:
start = time.time()

# Train on the CPU
x_history = Z.LINREG_SGD(num_epochs=num_epochs, step_size=step_size, regularize=0)

print('Performed linear regression on cadata. Training time: ' + str(time.time()-start))
# Print losses after each epoch
initial_loss = Z.calculate_LINREG_loss(np.zeros(Z.num_features), 0)
print('Initial loss: ' + str(initial_loss))
for e in range(0, num_epochs):
	loss = Z.calculate_LINREG_loss(x_history[:,e], 0)
	print('Epoch ' + str(e) + ' loss: ' + str(loss) )

Performed linear regression on cadata. Training time: 110.11404371261597
Initial loss: 0.384345112242
Epoch 0 loss: 0.00861334713324
Epoch 1 loss: 0.00805644661588
Epoch 2 loss: 0.00771162294714
Epoch 3 loss: 0.00746739865464
Epoch 4 loss: 0.00728293887823
Epoch 5 loss: 0.0071388712119
Epoch 6 loss: 0.00702395660503
Epoch 7 loss: 0.0069308542541
Epoch 8 loss: 0.00685445351566
Epoch 9 loss: 0.00679106218299


## 4. Full-precision training using the FPGA

Now, let's use the ZipML-PYNQ overlay to perform the same training process on the FPGA. We just replace the CPU-training function with the FPGA one. The training is 2 order of magnitudes faster on the FPGA.

In [6]:
start = time.time()

# Train on FPGA
Z.configure_SGD_FPGA(num_epochs, step_size, -1, -1, 0, 0)
x_history = Z.SGD_FPGA(num_epochs)

print('FPGA train time: ' + str(time.time()-start) )
# Print losses after each epoch
initial_loss = Z.calculate_LINREG_loss(np.zeros(Z.num_features), 0)
print('Initial loss: ' + str(initial_loss))
for e in range(0, num_epochs):
	loss = Z.calculate_LINREG_loss(x_history[:,e], 0)
	print('Epoch ' + str(e) + ' loss: ' + str(loss) )

Sent
Config Received
0x20
0x0
0xbf800000
0xbf800000
0x0
0x5b
0xc350
0xa
0x39800000
self.data_start_index: 0, self.data_end_index: 4700000
bytes_for_model: 368
FPGA train time: 0.330078125
Initial loss: 0.384345112242
Epoch 0 loss: 0.00862058925779
Epoch 1 loss: 0.00806469953043
Epoch 2 loss: 0.00772012776682
Epoch 3 loss: 0.00747571889878
Epoch 4 loss: 0.00729096349781
Epoch 5 loss: 0.00714612406087
Epoch 6 loss: 0.00703085683972
Epoch 7 loss: 0.00693696086218
Epoch 8 loss: 0.00685999522279
Epoch 9 loss: 0.00679613550255


## 5. Low-precision training using the FPGA

With the ZipML-PYNQ overlay, we can even improve upon previous results by using low-precision data. We call the quantization function that compresses the features of the data set using deterministic quantization. The rest of the code remains exactly the same as the previous example. We can see that the total training time is improved upon the full precision variant, while the convergence quality is kept. You can experiment with different precisions and most of the time you might see that 8-bits are enough to get to the same quality.

In [7]:
Z.a_quantize(quantization_bits=8)

start = time.time()

# Train on FPGA
Z.configure_SGD_FPGA(num_epochs, step_size, -1, -1, 0, 0)
x_history = Z.SGD_FPGA(num_epochs)

print('FPGA train time: ' + str(time.time()-start) )
# Print losses after each epoch
initial_loss = Z.calculate_LINREG_loss(np.zeros(Z.num_features), 0)
print('Initial loss: ' + str(initial_loss))
for e in range(0, num_epochs):
	loss = Z.calculate_LINREG_loss(x_history[:,e], 0)
	print('Epoch ' + str(e) + ' loss: ' + str(loss) )

num_levels: 129
qFSGD8.bit is loaded
num_quantized_items_in_word: 4.0
mask: 255
self.total_size: 1300000
Sent
Config Received
0x20
0x0
0xbf800000
0xbf800000
0x0
0x5b
0xc350
0xa
0x39800000
self.data_start_index: 4700000, self.data_end_index: 6000000
bytes_for_model: 448
FPGA train time: 0.11707735061645508
Initial loss: 0.384345112242
Epoch 0 loss: 0.00857622252748
Epoch 1 loss: 0.00800754022974
Epoch 2 loss: 0.00766088894158
Epoch 3 loss: 0.00741871949842
Epoch 4 loss: 0.00723901676682
Epoch 5 loss: 0.00710134055793
Epoch 6 loss: 0.0069939509613
Epoch 7 loss: 0.00690964300533
Epoch 8 loss: 0.00684188243595
Epoch 9 loss: 0.00678810862973
